In [45]:
import pandas as pd
import json

file_path = 'api_responses/STIF_StopArea_SP_62892__STIF_Line__C01743__20231129_210201.json'

try:
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    # Process the data here
except json.JSONDecodeError as e:
    print(f"Error decoding JSON in file: {e}")
    # Handle the error or take appropriate action
except FileNotFoundError:
    print(f"File not found: {file_path}")
    # Handle the file not found error or take appropriate action
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    # Handle other unexpected errors

In [46]:
# Extracting relevant data
service_delivery = data["Siri"]["ServiceDelivery"]
monitored_stop_visits = service_delivery["StopMonitoringDelivery"][0]["MonitoredStopVisit"]

# Flattening nested structures
df = pd.json_normalize(monitored_stop_visits, sep='_')
# Flattening nested structures
df['ResponseTimestamp'] = service_delivery['ResponseTimestamp']

In [48]:
# Replace specified strings in column names
str_replace = ['FramedVehicleJourneyRef_', 'MonitoredVehicleJourney_', 'MonitoredCall_', '_value']

for i in str_replace: 
    df.columns = df.columns.str.replace(i, '')

In [51]:
# Drop columns
columns_to_drop = ['DataFrameRef', 'ArrivalStatus', 'TrainNumbers_TrainNumberRef']
df = df.drop(columns_to_drop, axis=1)

In [55]:
# Convert date columns to datetime format
date_columns = ["RecordedAtTime", "ExpectedArrivalTime", "ExpectedDepartureTime", "ResponseTimestamp"]
df[date_columns] = df[date_columns].apply(pd.to_datetime)

In [61]:
# Convert date columns to GMT+1
gmt_date_columns = ["RecordedAtTime", "ExpectedArrivalTime", "ExpectedDepartureTime"]
df[gmt_date_columns] = df[gmt_date_columns].apply(lambda x: x.dt.tz_convert('Europe/Paris'))